<a href="https://colab.research.google.com/github/omkar806/Assignments/blob/main/Swipe_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pypdf openai tiktoken pdf2image
!pip install google-generativeai
!apt-get install -y poppler-utils

In [14]:
from pdf2image import convert_from_path
import google.generativeai as genai
from google.colab import userdata
import os
import imghdr
import PIL.Image

In [7]:
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GOOGLE_API_KEY = input("Please enter your Gemini API Key from Google AI studio : ")
genai.configure(api_key=GOOGLE_API_KEY)

In [9]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [44]:
def system_prompt()->str:
    system_prompt = f"""You are a Invoice/Receipt Analysing tool . Anaylse the image of the invoice provided and extract information from the following receipt image and return a JSON object with these exact keys:Customer_Details,Products,Total Amount.\n
    Rules:
    1. For total_cost, use the highest monetary value in the text.
    2. Customer_Details will consist of another json object consisting the keys Customer_Name,Customer_Address,Email Address,Phone Number,Customer ID,Billing Address,Shipping Address,Account Number,Tax ID/VAT Number,Company Name,Payment Method.
    3.Products will consist of another json object consisting of keys Product_name_1,quantity,unit_price.
    6. If any value is not found, return null.
    7. If all values are null, return null.
    Ensure the strictly that output is a valid JSON object containing strictly the above keys, without any explanations.
    Generate a JSON response in the following format without using the ```json block. Ensure the output is properly formatted as plain text JSON.\n
    \n
    """
    return system_prompt

In [39]:
#Function to generate a json with required keys
def get_invoice_details(image):
  response = model.generate_content([system_prompt(), image], stream=True)
  response.resolve()
  print(response.text)


In [42]:
def save_pdf_as_image(images):
  imgs=[]
  for i, image in enumerate(images):
    image.save(f'page_{i+1}.png', 'PNG')
    imgs.append(f'/content/page_{i+1}.png')
  return imgs

In [45]:
files_list = ['/content/Sample Invoice image.png','/content/Sample Invoice.pdf','/content/c669abb4-f485-4880-8973-cc7fdfeee22e.pdf']
images = []
# Iterate over the files and process them
for file in files_list:
  print(file)
  if imghdr.what(file):
    img = PIL.Image.open(file)
    print("pinting image")
    print(img)
    images.append(img)
  elif file.lower().endswith('.pdf'):
    img = convert_from_path(file)
    print("converted pdf to image")
    print(img)
    pdf_2_img=save_pdf_as_image(images)
    for pdfimg in pdf_2_img:
      imgg = PIL.Image.open(pdfimg)
      images.append(imgg)

  else:
    print(f"{file} is neither an image nor a PDF.")

for img in images:
  print("Initiating the task...")
  get_invoice_details(img)
  print("\n")







/content/Sample Invoice image.png
pinting image
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=907x962 at 0x7D17BF4E9FC0>
/content/Sample Invoice.pdf
converted pdf to image
[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339 at 0x7D17BF4E95D0>]
/content/c669abb4-f485-4880-8973-cc7fdfeee22e.pdf
converted pdf to image
[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339 at 0x7D17BF4EAA10>]
Initiating the task...
{
"Customer_Details": {
"Customer_Name": "Test",
"Customer_Address": "Hyderabad, TELANGANA, 500089",
"Email Address": "test@gmail.com",
"Phone Number": "9108239284",
"Customer ID": null,
"Billing Address": null,
"Shipping Address": null,
"Account Number": "1234567890",
"Tax ID/VAT Number": null,
"Company Name": null,
"Payment Method": null
},
"Products": {
"Product_name_1": "WASTE AND SCRAP OF STAINLESS STEEL",
"quantity": "6,790 KGS",
"unit_price": "95.00"
},
"Total Amount": "7,68,771.00"
}


Initiating the task...
{
"Customer_Details": {
"Customer_